In [7]:
import sys
sys.path.append("/home/pablo/tesis/tsm-pipeline/code")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from scripts.xxx import run_tsm, ExperimentSettings
import docker

docker_client = docker.from_env()
defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
    "o11y_db_dir": "/home/pablo/tesis/tsm-pipeline/code/tsm.db",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

## Prueba para ver si un ejemplo pequeño funciona en la Dell

El experimento de abajo es una versión de prueba para ver tanto que funciona bien la versión dockerizada y su wrapper en python, como tainted path.

In [8]:
test_experiment = ExperimentSettings(
    name="dell_test_run",
    query_type="path",
    project_list_file="/home/pablo/tesis/tsm-pipeline/experiments/tesis/tainted_path/test_4.txt",
    results_dir="/home/pablo/results/dell_test_run",
    **defaults,
)

run_tsm(docker_client, test_experiment, block=True)

2023-04-16 12:40:58,305 - INFO - creating /home/pablo/results/dell_test_run dir. Omitting error if already existing
2023-04-16 12:40:58,307 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-04-16 12:40:58,955 - INFO - running at container af62f572f79cca382139a0d48d301bbde2b7c7b566e528331947c0f61eec46c2. Use `docker logs af62f572f79cca382139a0d48d301bbde2b7c7b566e528331947c0f61eec46c2 --tail 10 --follow` to follow progress
2023-04-16 13:00:06,659 - INFO - running combine scores
g: /home/pablo/results/dell_test_run/*/TaintedPathWorse-*/reprScores.txt


In [9]:
!cat /home/pablo/results/dell_test_run/averaged-results.csv

"(member currentFormPage (parameter 1 (member render *)))","snk",0.2500000000
"(member error (parameter 0 (member send *)))","snk",0.5000000000
"(member form (parameter 1 (member render *)))","snk",0.2500000000
"(member page (parameter 1 (member render *)))","snk",0.2500000000
"(parameter 0 (member accessSync *))","snk",1.0000000000
"(parameter 0 (member appendFile *))","snk",1.0000000000
"(parameter 0 (member appendFileSync *))","snk",1.0000000000
"(parameter 0 (member createReadStream *))","snk",1.0000000000
"(parameter 0 (member createWriteStream *))","snk",1.0000000000
"(parameter 0 (member define (global)))","snk",1.0000000000
"(parameter 0 (member exists *))","snk",1.0000000000
"(parameter 0 (member existsSync *))","snk",1.0000000000
"(parameter 0 (member mkdirSync *))","snk",1.0000000000
"(parameter 0 (member open *))","snk",1.0000000000
"(parameter 0 (member readFile *))","snk",1.0000000000
"(parameter 0 (member readFileSync *))","snk",1.0000000000
"(parameter 0 (member readdir

Por lo visto, para las 4 DBs con las que corrio el experimento, que son:

In [12]:
!cat ../tesis/tainted_path/test_4.txt

ASCOT/dashboardjs/352a8b6710d6f2ddfd42ddfa0266af65c9c23ebe
alphagov/submit-prototype-kit/6ce9e75078ea7965cb9429446cca8c5765817d7c
RuffApps/Apps/90e13d23b0593aa45f35fdac736b8e19c6616a9d
Sv443/JokeAPI/f2c757a20bdc385edcf57b811ec8cc1a72899432

La primera dio timeout, pero el resto corrio bien y dentro de 4 minutos.

## Evaluando un pipeline completo, NoSQL

Entrenando con el top 100 de los repos de NoSQL de ATM. Tomo como corte de entrenamiento los primeros 70, y luego los siguientes 30 para evaluar.

Primero, para descargar y compilar todas las DBs:
```bash
# Download and compile 
python -m database.cli  --list ../experiments/tesis/atm_nosql_top100.txt --cache-root /home/pablo/dbcache --cli-version 2.5.2,2.10.5 --parallel --thread-count=4

# Check
python -m database.cli  --list ../experiments/tesis/atm_nosql_top100.txt --cache-root /home/pablo/dbcache --cli-version 2.5.2,2.10.5 --check
```

Luego, se puede ejecutar la fase de entrenamiento, produciendo los resultados en el directorio `results_dir` proveido:

In [14]:
nosql_atm_top_100 = ExperimentSettings(
    name="nosql_atm_top_100",
    query_type="nosql",
    project_list_file="/home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100_train.txt",
    results_dir="/home/pablo/results/nosql_atm_top100",
    **defaults
)

run_tsm(docker_client, nosql_atm_top_100, block=True)

2023-04-16 13:45:57,232 - INFO - creating /home/pablo/results/nosql_atm_top100 dir. Omitting error if already existing
2023-04-16 13:45:57,234 - INFO - creating /tmp/log dir. Omitting error if already existing
2023-04-16 13:45:58,080 - INFO - running at container 6afd831f775aa2bb5d10651ac67db3f1531f57ef7e508c2c88e9fc600eb62ad3. Use `docker logs 6afd831f775aa2bb5d10651ac67db3f1531f57ef7e508c2c88e9fc600eb62ad3 --tail 10 --follow` to follow progress
2023-04-16 14:56:31,054 - INFO - running combine scores
g: /home/pablo/results/nosql_atm_top100/*/NosqlInjectionWorse-*/reprScores.txt


In [15]:
!cat /home/pablo/results/nosql_atm_top100/averaged-results.csv|wc -l

57


In [16]:
from scripts.evaluate import evaluate, EvaluationSettings

In [17]:
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql",
    cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file="/home/pablo/results/nosql_atm_top100/averaged-results.csv",
)

evaluate(
    settings=worse_settings,
    output_dir="/home/pablo/results/nosql_atm_top100/worse",
    dbs_path="/home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100_test.txt",
    **defaults_evalute,
)

2023-04-16 15:32:53,999 - INFO - creating /home/pablo/results/nosql_atm_top100/worse dir. Omitting error if already existing
2023-04-16 15:32:54,713 - INFO - Starting processing with 4 processes
2023-04-16 15:32:54,820 - INFO - Evaluating amejiarosario - meanshop
2023-04-16 15:32:54,820 - INFO - Evaluating 3dbeb41841bfbfcc24d55143816cf7f1 - atl-wdi-10
2023-04-16 15:32:54,820 - INFO - Evaluating aheckmann - mquery
2023-04-16 15:32:54,820 - INFO - Evaluating capacitorset - rebridge
2023-04-16 15:36:48,731 - INFO - Evaluating createdd - votingapp
2023-04-16 15:36:59,852 - INFO - Evaluating cablej - race-condition-demo
2023-04-16 15:37:01,638 - INFO - Evaluating cleverbeagle - pup
2023-04-16 15:37:41,987 - INFO - Evaluating bhdouglass - uappexplorer
2023-04-16 15:37:47,390 - INFO - Evaluating bertrand31 - monitaure
2023-04-16 15:37:52,473 - INFO - Evaluating catsmiaow - tested
2023-04-16 15:38:30,491 - INFO - Evaluating cyantarek - django-microservices
2023-04-16 15:38:33,885 - INFO - Eval

In [18]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.10.5",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
)

evaluate(
    settings=v0_settings,
    output_dir="/home/pablo/results/nosql_atm_top100/v0",
    dbs_path="/home/pablo/tesis/tsm-pipeline/experiments/tesis/atm_nosql_top100_test.txt",
    **defaults_evalute,
)

2023-04-16 16:14:17,008 - INFO - creating /home/pablo/results/nosql_atm_top100/v0 dir. Omitting error if already existing
2023-04-16 16:14:18,396 - INFO - Starting processing with 4 processes
2023-04-16 16:14:18,508 - INFO - Evaluating capacitorset - rebridge
2023-04-16 16:14:18,508 - INFO - Evaluating 3dbeb41841bfbfcc24d55143816cf7f1 - atl-wdi-10
2023-04-16 16:14:18,503 - INFO - Evaluating amejiarosario - meanshop
2023-04-16 16:14:18,508 - INFO - Evaluating aheckmann - mquery
2023-04-16 16:19:23,187 - INFO - Evaluating cablej - race-condition-demo
2023-04-16 16:19:23,018 - INFO - Evaluating createdd - votingapp
2023-04-16 16:19:23,308 - INFO - Evaluating cleverbeagle - pup
2023-04-16 16:19:24,175 - INFO - Evaluating biaochenxuying - blog-node
2023-04-16 16:20:02,653 - INFO - Evaluating bhdouglass - uappexplorer
2023-04-16 16:20:05,338 - INFO - Evaluating bertrand31 - monitaure
2023-04-16 16:20:05,621 - INFO - Evaluating catsmiaow - tested
2023-04-16 16:20:14,603 - INFO - Evaluating ac

In [19]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder="/home/pablo/results/nosql_atm_top100",
)

2023-04-16 16:52:02,977 - INFO - Result sets sizes: Worse 260, Boosted 115, V0 108, All 368
2023-04-16 16:52:03,008 - INFO - Score results: Precision: 0.0000. Recall: 0.0000. Accuracy: 0.3940


(0.0, 0.0, 0.39402173913043476)

Los scores excepto por accuracy son pesimos. Necesito debuggear si es un problema de cómo calculo los scores, cómo evalúo, o si posta la técnica está dando resultados muy chotos. Quizas también puede que sean las dbs.